## CuPy: A GPU-Accelerated Drop-In Replacement for NumPy

CuPy is a library that mirrors NumPy's functionality but utilizes the GPU for computations, offering an almost seamless way to accelerate Python code.

Like NumPy, CuPy provides three core components:

1. A multidimensional array object (stored in GPU memory)
2. A `ufunc` system that follows broadcasting rules (executing operations in parallel on the GPU)
3. A comprehensive library of array functions (implemented with CUDA for efficient GPU execution)

One of CuPy's main strengths is its ability to serve as a drop-in replacement for NumPy, allowing us to write "agnostic code" that can run on either CPU or GPU, depending on the available hardware.

In [ ]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt

### Basic Operations with CuPy (Comparing with NumPy)

Let's start with a few basic operations using CuPy, similar to how we would with NumPy.

In [ ]:
# Vector sum with NumPy
x_cpu = np.linspace(0, 100, 20)
y_cpu = np.linspace(10, 200, 20)
z_cpu = x_cpu + y_cpu
z_cpu 

In [ ]:
# Matrix-matrix multiplication with NumPy (CPU)
x_cpu = np.random.random(1_000).reshape(20, 50)
y_cpu = np.random.random(1_000).reshape(50, 20)
z_cpu = np.dot(x_cpu, y_cpu)
z_cpu 

To perform the same operations with CuPy, we simply need to change the library import from `numpy` to `cupy`. This allows us to run the operations on the GPU instead of the CPU, with minimal code modification.

In [ ]:
# Vector sum on the GPU using CuPy
x_gpu = cp.linspace(0, 100, 20)
y_gpu = cp.linspace(10, 200, 20)
z_gpu = x_gpu + y_gpu
z_gpu 

In [ ]:
# Matrix-matrix multiplication on the GPU using CuPy
x_gpu = cp.random.random(1_000).reshape(20, 50)
y_gpu = cp.random.random(1_000).reshape(50, 20)
z_gpu = cp.dot(x_gpu, y_gpu)
z_gpu

Unlike NumPy's `numpy.ndarray` objects, CuPy arrays are represented as `cupy.ndarray`. 

By default, CuPy _infers_ the data type for you, but it may not always choose the optimal format for your specific use case. It's a good practice to explicitly define the data type when necessary to ensure optimal performance and memory usage.

In [ ]:
# Checking the type, data type, and shape of the NumPy array
print(type(z_cpu))     # Type of the array
print(z_cpu.dtype)     # Data type of the elements in the array
print(z_cpu.shape)     # Shape of the array

In [ ]:
# Checking the type, data type, shape, and device of the CuPy array
print(type(z_gpu))       # Type of the array
print(z_gpu.dtype)       # Data type of the elements in the array
print(z_gpu.shape)       # Shape of the array
print(z_gpu.device)      # Device where the array is allocated (GPU)

All operations executed with CuPy incur some overhead in terms of execution time. This overhead arises from the time required to compile the code into CUDA and transfer it for execution on the GPU.

While memory management in CuPy is handled automatically, it may not always perform as efficiently as desired. However, you have the ability to control data transfers to and from the GPU as needed.

In [ ]:
# Create a NumPy array on the host and transfer it to the device (GPU)
a_cpu = np.array([0, 1, 2, 3, 4, 5])
a_gpu = cp.asarray(a_cpu)  # Transfer to GPU

In [ ]:
# Perform a computation on the CuPy array on the device (GPU)
b_gpu = cp.exp(a_gpu.reshape(2, 3))  # Calculate the exponential of each element after reshaping

In [ ]:
# Copy the device data back to the host (CPU)
b_cpu = b_gpu.get()  # Transfer the data from GPU to CPU
b_cpu

## Agnostic Code

Thanks to the CuPy team's effort in creating a 1-to-1 mapping of the NumPy APIs, one of the key advantages is the ability to write the same function and use it interchangeably on either the CPU or GPU.

CuPy also provides the capability to identify array types, enabling us to write device-agnostic code. This means we can pass an array (whether it's a NumPy or CuPy array) to our custom function and let the interpreter decide which backend to use, based on the location of the array (CPU or GPU).

In [ ]:
# Agnostic function implementation 
def softplus(x):
    # Infer whether the array is a NumPy or CuPy array
    # `xp` will be set to `cp` if x is on the GPU, or `np` if x is on the CPU
    xp = cp.get_array_module(x)
    print("Using:", xp.__name__)  # Display the library in use
    return xp.maximum(0, x) + xp.log1p(xp.exp(-abs(x)))  # Softplus computation

In [ ]:
# Create a NumPy array on the CPU and transfer it to the GPU
x_cpu = np.random.random(10_000)  # Generate a random array with 10,000 elements
x_gpu = cp.asarray(x_cpu)  # Transfer the NumPy array to the GPU as a CuPy array

In [ ]:
# Apply the softplus function on the NumPy array (CPU)
result_cpu = softplus(x_cpu)  # Compute softplus for the CPU array
result_cpu

In [ ]:
# Apply the softplus function on the CuPy array (GPU)
result_gpu = softplus(x_gpu)  # Compute softplus for the GPU array
result_gpu

## Embedded Benchmarking

CuPy includes a built-in profiler that facilitates the creation and management of all `cuda.Event` objects necessary for measuring execution time on the device. This allows you to easily benchmark the performance of your GPU computations.

In [ ]:
# Import the CuPy profiler for benchmarking
# (`cupyx` is the module including the cupy-specific functionalities)
from cupyx.profiler import benchmark

In [ ]:
# Benchmark the softplus function on the CPU
cpu_bench = benchmark(softplus, (x_cpu,), n_repeat=10)  # Repeat the benchmark 10 times
print(cpu_bench)

In [ ]:
# Benchmark the softplus function on the GPU
gpu_bench = benchmark(softplus, (x_gpu,), n_repeat=10)  # Repeat the benchmark 10 times
print(gpu_bench)  # Display the benchmark results

## User-Defined Kernels

CuPy provides three types of CUDA kernel definitions:
- Elementwise
- Reduction
- Raw

### Elementwise Kernels

Elementwise kernels are functions or operations that are applied independently to each element of one or more input arrays. These operations are executed simultaneously in parallel across multiple data elements.

This concept is similar to the `vectorized` or `guvectorized` functions we've encountered with Numba and CUDA.

The definition of an elementwise kernel consists of four parts:
1. An input argument list.
2. An output argument list.
3. The kernel body code.
4. The kernel name.

In [ ]:
# Define an elementwise kernel using CuPy
kernel = cp.ElementwiseKernel(
    'float32 x, float32 y',             # Input argument list
    'float32 z',                        # Output argument list
    '''if (x - 2 > y) { z = x * y; }     
    else { z = x + y; }''',             # Kernel body code (executed on each thread)
    'elemwise_kernel'                   # Name of the kernel
)

In [ ]:
# Create two CuPy arrays for kernel input
x = cp.arange(6, dtype='float32').reshape(2, 3)  # 2x3 array with float32 data type
y = cp.arange(3, dtype='float32')                # 1D array with 3 elements and float32 data type

In [ ]:
# Benchmark the elementwise kernel using CuPy
kernel_bench = benchmark(kernel, (x, y), n_repeat=10)  # Repeat the benchmark 10 times
print(kernel_bench)

In [ ]:
# Execute the elementwise kernel on the input arrays and retrieve the result
z = kernel(x, y)  # Apply the kernel
result = z.get()  # Transfer the result from GPU to CPU
result

The same kernel, which is currently defined to operate specifically on `float32` arrays, can be made generic to support arbitrary data types. This allows the data type to be determined at compile time, enabling greater flexibility and reusability of the kernel for different types of input arrays.

In [ ]:
# Define a generic type elementwise kernel using CuPy
kernel_gtype = cp.ElementwiseKernel(
    'T x, T y',                          # Input arguments of type T
    'T z',                               # Output argument of the same type T
    '''if (x - 2 > y) { z = x * y; }     
    else { z = x + y; }''',              
    'elemwise_kernel_generic_type'      # Name of the kernel
)

In [ ]:
# Create two CuPy arrays with integer data types and execute the generic type kernel
x = cp.arange(6, dtype='int32').reshape(2, 3)  # 2x3 array of int32
y = cp.arange(3, dtype='int32')                 # 1D array of int32

# Apply the generic type kernel
z = kernel_gtype(x, y)  
result_generic = z.get()  # Transfer the result from GPU to CPU
result_generic

### Reduction Kernels

Reduction kernels are functions or operations that combine multiple elements of an input array into a single result by applying a reduction operation, such as summing, finding the minimum or maximum, or calculating the average of the elements. These kernels are particularly useful for efficiently computing global aggregates or statistics from large arrays.

The definition of a reduction kernel consists of several components:
1. **Identity Value**: This value serves as the initial value for the reduction process.
2. **Mapping Expression**: This is used for the pre-processing of each element to be reduced.
3. **Reduction Expression**: This operator reduces multiple mapped values, using the special variables `a` and `b` as its operands.
4. **Post Mapping Expression**: This transforms the resulting reduced values, using the special variable `a` as input. The output should be written to the output parameter.

In [ ]:
# Define a reduction kernel for computing the L2 norm using CuPy
l2norm_kernel = cp.ReductionKernel(
    'T x',              # Input parameters
    'T y',              # Output parameters
    'x * x',            # Mapping expression: squares each element
    'a + b',            # Reduction expression: sums the mapped values
    'y = sqrt(a)',      # Post-reduction mapping: takes the square root of the sum
    '0',                # Identity value for the reduction
    'l2norm'            # Name of the kernel
)

In [ ]:
# Create a CuPy array and compute the L2 norm using the reduction kernel
x = cp.arange(10, dtype=np.float32).reshape(2, 5)  # 2x5 array of float32
l2norm_result = l2norm_kernel(x, axis=1)  # Compute L2 norm along axis 1
l2norm_result_host = l2norm_result.get()
l2norm_result_host

### Raw Kernels (The Good Old CUDA-C Kernels...)

With raw kernels, we can define kernels directly from raw CUDA source code.

Much like Numba+CUDA, this approach bridges the gap between Python and CUDA-C, allowing us to leverage the exact CUDA-C functions we've previously declared in our Python code.

This means we can build the majority of our codebase using Python and NumPy, accelerating specific parts as needed by using CuPy as a drop-in replacement. We can then call CUDA-C functions—and even external CUDA-C libraries—while remaining within the same Python interpreter.

In [ ]:
# Define a raw CUDA kernel for matrix multiplication using CuPy
custom_kernel = cp.RawKernel(r'''
    extern "C" __global__ 
    void naiveMatrixMultiplication(const float* M, const float* N, float* P, const int width) {
        // Calculate the thread ID within the overall grid
        int row = blockIdx.y * blockDim.y + threadIdx.y;
        int col = blockIdx.x * blockDim.x + threadIdx.x;

        // Each thread computes one element of the result matrix
        if (row < width && col < width) {
            float sum = 0.0;
            // Access all elements of a row of M and a column of N
            for (int k = 0; k < width; ++k) {
                sum += M[row * width + k] * N[k * width + col];
            }
            P[row * width + col] = sum;  // Store the result
        }
    }
    ''',
    'naiveMatrixMultiplication')  # Name of the kernel

In [ ]:
# Set the dimensions for the matrices and create them
width = 2048  # Size of the square matrices
M = cp.random.random((width, width), 'float32')  # Random matrix M of size 2048x2048
N = cp.random.random((width, width), 'float32')  # Random matrix N of size 2048x2048
P = cp.zeros_like(N)  # Result matrix P initialized to zeros with the same shape as N

In [ ]:
# Define the number of threads per block and calculate the number of blocks needed
threads_per_block = (32, 32)  # Each block will have 32x32 threads
blocks = (int((width + threads_per_block[0] - 1) / threads_per_block[0]), 
           int((width + threads_per_block[1] - 1) / threads_per_block[1]))  # Calculate blocks needed

print("Threads per block:", threads_per_block)  # Output the thread configuration
print("Number of blocks:", blocks)  # Output the calculated number of blocks

In [ ]:
# Launch the kernel
custom_kernel(blocks, threads_per_block, (M, N, P, width))
# Retrieve the P values
P.get()

In [ ]:
# Benchmark the raw CUDA kernel for matrix multiplication
kernel_bench = benchmark(custom_kernel, (blocks, threads_per_block, (M, N, P, width)), n_repeat=5)  # Repeat the benchmark 5 times
print(kernel_bench)  

In [ ]:
# Printing P will result in a call to the device memory location (an implicit `get`)
P

In [ ]:
# Define a function for matrix multiplication using CuPy's dot product
def cp_matmul(M, N):
    return cp.dot(M, N)  # Multiply matrices M and N using CuPy's dot function

In [ ]:
# Benchmark the CuPy matrix multiplication function
cp_matmul_bench = benchmark(cp_matmul, (M, N), n_repeat=5)  # Repeat the benchmark 5 times
print(cp_matmul_bench)  

In [ ]:
# Perform matrix multiplication using the CuPy-defined function
P_cp = cp_matmul(M, N)  # Multiply matrices M and N, storing the result in P_cp

In [ ]:
# Check if the result from CuPy's matrix multiplication is close to the expected result
is_close = np.allclose(P, P_cp)  # Verify that all elements in P and P_cp are close within a tolerance
is_close  

## Advanced Algebraic and Scientific Applications Made Simple

Most of the algebraic functionalities from NumPy, as well as some from SciPy (though not all; please check the documentation for details), are included in CuPy's library of rewritten CUDA kernels that operate on CuPy inputs.

* [Reference of NumPy routines included in CuPy](https://docs.cupy.dev/en/stable/reference/routines.html)
* [Reference of SciPy routines included in CuPy](https://docs.cupy.dev/en/stable/reference/scipy.html)
* [An extremely useful comparison between NumPy and CuPy](https://docs.cupy.dev/en/stable/reference/comparison.html)

### Algebraic functions

In [ ]:
# Perform singular value decomposition (SVD) using NumPy
x_cpu = np.random.random((1000, 1000))  # Generate a random 1000x1000 matrix
u, s, v = np.linalg.svd(x_cpu)  # Compute the SVD of the matrix, resulting in U, singular values S, and V

In [ ]:
# Perform singular value decomposition (SVD) using CuPy
x_gpu = cp.asarray(x_cpu)  # Use the same inputs as before
u_cp, s_cp, v_cp = cp.linalg.svd(x_gpu)  # Compute the SVD of the matrix on the GPU, resulting in U, singular values S, and V

### Fitting and evaluating functions

In [ ]:
# Generate noisy data for polynomial fitting
x = np.linspace(0, 10, 100)  # Create an array of 100 points from 0 to 10
y_true = 2 * np.sin(x) + 0.5 * x  # Define the true relationship
noise = np.random.normal(0, 0.5, x.shape)  # Generate Gaussian noise
y = y_true + noise  # Create the noisy observations

# Perform polynomial fitting using numpy.polyfit()
degree = 5  # Define the degree of the polynomial
coeffs = np.polyfit(x, y, degree)  # Fit a polynomial of the specified degree to the data

# Generate predictions using the fitted polynomial
y_pred = np.polyval(coeffs, x)  # Evaluate the polynomial at the points in x

In [ ]:
# Plot the noisy data and the polynomial fit
plt.plot(x, y, '.', label='Data')  # Scatter plot of the noisy data
plt.plot(x, y_pred, '-', label='Fit')  # Line plot of the fitted polynomial
plt.legend(loc='best')  # Display the legend in the best location
plt.xlabel('X')  # Label for the x-axis
plt.ylabel('Y')  # Label for the y-axis
plt.show()  # Display the plot

In [ ]:
# Generate noisy data on the GPU using CuPy
x_gpu = cp.asarray(x)  # Convert the x array to a CuPy array
y_gpu = cp.asarray(y)  # Convert the y array to a CuPy array

# Perform polynomial fitting using cupy.polyfit()
degree = 5  # Define the degree of the polynomial
coeffs_cp = cp.polyfit(x_gpu, y_gpu, degree)  # Fit a polynomial of the specified degree to the data on the GPU

# Generate predictions using the fitted polynomial
y_pred_cp = cp.polyval(coeffs_cp, x_gpu)  # Evaluate the polynomial at the points in x using CuPy

In [ ]:
# Plot the noisy data and the polynomial fit
plt.plot(x, y, '.', label='Data')  # Scatter plot of the noisy data
plt.plot(x, y_pred, '.-', label='Fit (CPU)')  # Line plot of the fitted polynomial (CPU)
plt.plot(x, y_pred_cp.get(), '--', label='Fit (GPU)')  # Line plot of the fitted polynomial (GPU)
plt.legend(loc='best')  # Display the legend in the best location
plt.xlabel('X')  # Label for the x-axis
plt.ylabel('Y')  # Label for the y-axis
plt.show()  # Display the plot

### SciPy-equivalent functionalities

In [ ]:
# Create an image with repeated delta functions
deltas = np.zeros((2048, 2048))  # Initialize a 2048x2048 array filled with zeros
deltas[8::16, 8::16] = 1  # Set every 16th pixel starting from (8, 8) to 1, creating a grid of delta functions

In [ ]:
# Plot a zoomed-in version of the grid of delta functions
plt.imshow(deltas[0:200, 0:200])  # Display a 200x200 section of the array
plt.colorbar()  # Add a color bar to the side for reference
plt.show() 

In [ ]:
# Create a Gaussian filter
x, y = np.meshgrid(np.linspace(-2, 2, 15), np.linspace(-2, 2, 15))  # Create a mesh grid
dst = np.sqrt(x * x + y * y)  # Calculate the distance from the center
sigma = 1.0  # Standard deviation for the Gaussian
muu = 0.0  # Mean (center) of the Gaussian
gauss = np.exp(-((dst - muu) ** 2 / (2.0 * sigma ** 2)))  # Compute the Gaussian function

# Plot the Gaussian filter
plt.imshow(gauss)  # Display the Gaussian filter with a color map
plt.colorbar()  # Add a color bar to the side for reference
plt.show() 

In [ ]:
# Transfer the delta functions and Gaussian filter to the GPU using CuPy
deltas_gpu = cp.array(deltas)  # Convert the delta functions array to a CuPy array on the GPU
gauss_gpu = cp.array(gauss)  # Convert the Gaussian filter array to a CuPy array on the GPU

In [ ]:
# Load the equivalent CuPy function for 2D convolution
# (Equivalent in CuPy to the host code --> from scipy.signal import convolve2d)
from cupyx.scipy.signal import convolve2d  # Import the 2D convolution function from CuPy's SciPy module

# Apply the 2D convolution
convolved_img_gpu = convolve2d(deltas_gpu, gauss_gpu)  # Convolve the delta functions with the Gaussian filter on the GPU

In [ ]:
# Transfer the convolved image result from the GPU to the host (CPU)
convolved_img = convolved_img_gpu.get()  # Retrieve the convolved image as a NumPy array

In [ ]:
# Plot a zoomed-in version of the smeared grid after convolution
plt.imshow(convolved_img[0:200, 0:200])  # Display a 200x200 section of the convolved image
plt.colorbar()  # Add a color bar to the side for reference
plt.show() 